# Geocoding 1980 data, using the _Geographic Base File/Dual Independent Map Encoding (GBF/DIME), 1980_

This program uses the [Geographic Base File/Dual Independent Map Encoding (GBF/DIME), 1980](https://doi.org/10.3886/ICPSR08378.v1) data which were formatted using the code here: https://github.com/bleckley/GBF-DIME-1980

In [1]:
#------------------------------------------------------------------------------------
#
#         SECTION 1: Setting up all the resources needed for the later work
#
#------------------------------------------------------------------------------------

#Import necessary modules
import pandas as pd
import pandas_usaddress #Note: if not installed: pip install pandas-usaddress
import numpy as np


#Defining a function to determine if an address is on the right (even) or left (odd) side of the street
def side_function(row):
    if type(row['AddressNumber']) == float : 
        if np.isnan(row['AddressNumber']) :
            val = None
    elif row['AddressNumber'] is None :
        val = None
    elif row['AddressNumber'].strip()[-1] in ('0','2','4','6','8') :
        val = 'Right'
    elif row['AddressNumber'].strip()[-1] in ('1','3','5','7','9') :
        val = 'Left'
    else : val = 'Error'
    return val


#--------------Creating dictionary of state FIPS codes-----------------

state_fips = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46', 'AS': '60', 'GU': '66',
    'MP': '69', 'PR': '72', 'VI': '78'
}


#----------CREATING DATAFRAME FOR PLACE CODES ----------------------

#Reading text file from url to dataframe 
#Designating datatype to ensure all data are strings to retain leading zeros
place_df = pd.read_fwf('https://www2.census.gov/geo/tiger/PREVGENZ/pl/us_places.txt', dtype={'CENSUS': object, 'FIPS': object, 'NAME': object})

#Use regular expression to remove superfluous words at the end.
place_df.replace(' (?:city|town|village|CDP|comunidad|borough|zona urbana).*?$','',regex=True, inplace = True)

#Dropping unused column
place_df = place_df.drop(['FIPS'], axis=1)


#Creating State and Place columns from CENSUS column
place_df['STATE'] = place_df['CENSUS'].str[:2]
place_df['PLACE'] = place_df['CENSUS'].str[2:]

#Converting city names to uppercase
place_df['NAME'] = place_df['NAME'].str.upper()



#------------------------------------------------------------------------------------
#
#         SECTION 2: Creating the dataframe from address source file
#
#------------------------------------------------------------------------------------


#------------Loading the address data--------------------------

#Note: the needs of this section will change depending on the source file format and layout

#load dataframe
df = pd.read_csv('sourcefilename')
#df.columns = ['streetaddress', 'citystzip'] #--Assign column names if a text file without headers

#Parse to columns
df = pandas_usaddress.tag(df, ['Address', 'City', 'State', 'Zip'], granularity='medium', standardize=True)

#Convert string columns to upper case
df['PlaceName'] = df['PlaceName'].str.upper()
df['StateName'] = df['StateName'].str.upper()
df['StreetName'] = df['StreetName'].str.upper()
df['StreetNamePreDirectional'] = df['StreetNamePreDirectional'].str.upper()
df['StreetNamePostDirectional'] = df['StreetNamePostDirectional'].str.upper()
df['StreetNamePrefix'] = df['StreetNamePrefix'].str.upper()
df['StreetNameSuffix'] = df['StreetNameSuffix'].str.upper()

#Adding a unique ID so that later we can see records excluded (since index can change)
df.insert(0, 'New_ID', range(0, len(df)))


#-------------------------TEMP FIX-----------------------------------

#NOTE: I'm seeing errors in the StreetNameSuffix and PlaceName fields, 
#where City was parsed into "Ann" and "Arbor," respectively.
#I submitted an error report to the developers and will use the following work-around for now...

#Deleting erroneous "Ann" from StreetNameSuffix column.
df['StreetNameSuffix'].replace({'ANN': ''}, inplace = True)
#Recoding "ARBOR" to "ANN ARBOR" in PlaceName column.
df['PlaceName'].replace('ARBOR', 'ANN ARBOR', inplace = True)

#This may need to be expanded to other multi-word place names, including major cities like New York and Los Angeles.
#It will be important to spot-check the dataframe to see how those place names get parsed.

#------------------------------------------------------------


#Add State FIPS code to dataframe
df['STFIPS'] = df['StateName'].map(state_fips)

#Add Census place code to dataframe
df = df.merge(place_df, left_on=['STFIPS', 'PlaceName'], right_on=['STATE', 'NAME'], how='left')

#Dropping unneeded columns
#Note: could drop many more columns in the future to save space, esp when scaling up
#Leaving other columns for now to allow for trouble-shooting
df = df.drop(['CENSUS', 'STATE', 'NAME'], axis=1)

#Creating flag for side of the street (even houses numbers = Right)
df['StreetSide'] =  df.apply(side_function, axis=1)

#Changing the whole dataframe to string and just the address numbers to integers to ensure future merges work.
df = df.astype(str)
df = df.astype({'AddressNumber': 'int'})


#------------------------------------------------------------------------------------
#
#         SECTION 3: Loading the GBF/DIME file to a dataframe and geocoding source data
#
#------------------------------------------------------------------------------------


#--------------------Loading GBF/DIME file-----------------------

# NOTE: using Ann Arbor for testing...need to change to national file for full run
# 
gbf = pd.read_csv('08378-0013.csv', dtype={'STPREDIR': object, 'STNAME': object, 'STTYPE': object, 'STSUFDIR': object, 'NONSTCO': object, 'MAPNO1': object, 'MAPNO2': object, 'MAPNO3': object,
                                           'MAPNO4': object, 'COFLAG': object, 'LEFTADD1': object, 'LEFTADD2': object, 'RGTADD1': object, 'RGTADD2': object, 'CENTRA1': object, 'CENTRA2': object,
                                           'CENTRA3': object, 'CENTRA4': object, 'ZIPCOLEF': object, 'ZIPCORGT': object, 'SMSA': object, 'PLACO1': object, 'PLACO2': object, 'STCOLEFT': object, 
                                           'CNTCOLFT': object, 'MCDCCDL': object, 'BLKLEFT': object, 'STCORGT': object, 'CNTCORGT': object, 'MCDCCDR': object, 'BLOCKRGT': object, 
                                           'STPLACO1': object, 'STPLACO2': object, 'FROMLAT': object, 'FROMLONG': object, 'TOLAT': object, 'TOLONG': object, 'STPLA1': object, 'STPLA2': object,
                                           'STPLA3': object, 'STPLA4': object, 'LEFTADD3': object, 'LEFTADD4': object, 'RGTADD3': object, 'RGTADD4': object})

#removing the descriptive row of the dataframe...it's really not needed, messes up merges, and could be removed from the initial file setup
gbf = gbf.drop([0], axis=0)
#removing the rows without address numbers because they are not useful for this project and tend to be records of political boundaries, rivers, freeways, etc.
gbf = gbf.dropna(subset=['LEFTADD1', 'RGTADD1'])

#Need to recode Avenues because gecode file uses "AV" and source file uses "AVE"
#Depending on the source file, this might not be needed or might be more efficient to change the source file dataframe.
gbf['STTYPE'].replace({'AV': 'AVE'}, inplace = True)

#Changing the whole dataframe to string and just the address numbers to int to ensure future merges work.
gbf = gbf.astype(str)
gbf = gbf.astype({'LEFTADD1': 'int', 'LEFTADD2': 'int', 'RGTADD1': 'int', 'RGTADD2': 'int'})


#--------------------Merging the source data and the GBF/DIME data-----------------------

# Note: the GBF/DIME file is divided into sides of the street (since Census tracts often use streets as borders/boundaries)


#Geocoding the right side of the street first
dfr = df[df['StreetSide']=='Right']

#Since the GBF/DIME file uses address number ranges, numpy is used to merge the two data frames based on those ranges as well as Boolean comparisons
#(reference: https://stackoverflow.com/questions/44367672/best-way-to-join-merge-by-range-in-pandas)
addnum = dfr.AddressNumber.to_numpy()
maplow = gbf.RGTADD1.to_numpy()
maphigh = gbf.RGTADD2.to_numpy()

i, j = np.where((addnum[:, None] >= maplow) & (addnum[:, None] <= maphigh) 
                & (dfr.StreetName.to_numpy()[:, None] == gbf.STNAME.to_numpy())
                & (dfr.StreetNamePreDirectional.to_numpy()[:, None] == gbf.STPREDIR.to_numpy())
                & (dfr.StreetNameSuffix.to_numpy()[:, None] == gbf.STTYPE.to_numpy())
                & (dfr.StreetNamePostDirectional.to_numpy()[:, None] == gbf.STSUFDIR.to_numpy())
                & (dfr.PLACE.to_numpy()[:, None] == gbf.PLACO2.to_numpy())
                & (dfr.STFIPS.to_numpy()[:, None] == gbf.STCORGT.to_numpy()))

df_range_r = pd.DataFrame(
    np.column_stack([dfr.values[i], gbf.values[j]]),
    columns=dfr.columns.append(gbf.columns)).set_index(['New_ID'])


# Iterating the records that didn't match above, based on missing values in search keys


#Subset what records were not merged above
dfr_remain = dfr[~dfr.index.isin(df_range_r.index)]
# If there is no street type...
dfr_next = dfr_remain[dfr_remain['StreetNameSuffix']=='']
addnum = dfr_next.AddressNumber.to_numpy()

i, j = np.where((addnum[:, None] >= maplow) & (addnum[:, None] <= maphigh) 
                & (dfr_next.StreetName.to_numpy()[:, None] == gbf.STNAME.to_numpy())
                & (dfr_next.StreetNamePreDirectional.to_numpy()[:, None] == gbf.STPREDIR.to_numpy())
                & (dfr_next.StreetNamePostDirectional.to_numpy()[:, None] == gbf.STSUFDIR.to_numpy())
                & (dfr_next.PLACE.to_numpy()[:, None] == gbf.PLACO2.to_numpy())
                & (dfr_next.STFIPS.to_numpy()[:, None] == gbf.STCORGT.to_numpy()))

df_range_r = df_range_r.append(pd.DataFrame(
    np.column_stack([dfr_next.values[i], gbf.values[j]]),
    columns=dfr_next.columns.append(gbf.columns)).set_index(['New_ID']))

#Subset what records were not merged above
dfr_remain = dfr[~dfr.index.isin(df_range_r.index)]
# If there is no street direction prefix...
dfr_next = dfr_remain[dfr_remain['StreetNamePreDirectional']=='']
addnum = dfr_next.AddressNumber.to_numpy()

i, j = np.where((addnum[:, None] >= maplow) & (addnum[:, None] <= maphigh) 
                & (dfr_next.StreetName.to_numpy()[:, None] == gbf.STNAME.to_numpy())
                & (dfr_next.StreetNameSuffix.to_numpy()[:, None] == gbf.STTYPE.to_numpy())
                & (dfr_next.StreetNamePostDirectional.to_numpy()[:, None] == gbf.STSUFDIR.to_numpy())
                & (dfr_next.PLACE.to_numpy()[:, None] == gbf.PLACO2.to_numpy())
                & (dfr_next.STFIPS.to_numpy()[:, None] == gbf.STCORGT.to_numpy()))

df_range_r = df_range_r.append(pd.DataFrame(
    np.column_stack([dfr_next.values[i], gbf.values[j]]),
    columns=dfr_next.columns.append(gbf.columns)).set_index(['New_ID']))

# There is definitely room here to add code for other missing data or to add other data elements.

#---------------------------------------------------


#Repeating all of the above with the left side of the street now

dfl = df[df['StreetSide']=='Left']

addnum = dfl.AddressNumber.to_numpy()
maplow = gbf.LEFTADD1.to_numpy()
maphigh = gbf.LEFTADD2.to_numpy()


i, j = np.where((addnum[:, None] >= maplow) & (addnum[:, None] <= maphigh) 
                & (dfl.StreetName.to_numpy()[:, None] == gbf.STNAME.to_numpy())
                & (dfl.StreetNamePreDirectional.to_numpy()[:, None] == gbf.STPREDIR.to_numpy())
                & (dfl.StreetNameSuffix.to_numpy()[:, None] == gbf.STTYPE.to_numpy())
                & (dfl.StreetNamePostDirectional.to_numpy()[:, None] == gbf.STSUFDIR.to_numpy())
                & (dfl.PLACE.to_numpy()[:, None] == gbf.PLACO1.to_numpy())
                & (dfl.STFIPS.to_numpy()[:, None] == gbf.STCOLEFT.to_numpy()))

df_range_l = pd.DataFrame(
    np.column_stack([dfl.values[i], gbf.values[j]]),
    columns=dfl.columns.append(gbf.columns)).set_index(['New_ID'])


# Iterating the records that didn't match above, based on missing values in search keys


#Subset what records were not merged above
dfl_remain = dfl[~dfl.index.isin(df_range_l.index)]
# If there is no street type...
dfl_next = dfl_remain[dfl_remain['StreetNameSuffix']=='']
addnum = dfl_next.AddressNumber.to_numpy()

i, j = np.where((addnum[:, None] >= maplow) & (addnum[:, None] <= maphigh) 
                & (dfl_next.StreetName.to_numpy()[:, None] == gbf.STNAME.to_numpy())
                & (dfl_next.StreetNamePreDirectional.to_numpy()[:, None] == gbf.STPREDIR.to_numpy())
                & (dfl_next.StreetNamePostDirectional.to_numpy()[:, None] == gbf.STSUFDIR.to_numpy())
                & (dfl_next.PLACE.to_numpy()[:, None] == gbf.PLACO1.to_numpy())
                & (dfl_next.STFIPS.to_numpy()[:, None] == gbf.STCOLEFT.to_numpy()))

df_range_l = df_range_l.append(pd.DataFrame(
    np.column_stack([dfl_next.values[i], gbf.values[j]]),
    columns=dfl_next.columns.append(gbf.columns)).set_index(['New_ID']))

#Subset what records were not merged above
dfl_remain = dfl[~dfl.index.isin(df_range_l.index)]
# If there is no street direction prefix...
dfl_next = dfl_remain[dfl_remain['StreetNamePreDirectional']=='']
addnum = dfl_next.AddressNumber.to_numpy()

i, j = np.where((addnum[:, None] >= maplow) & (addnum[:, None] <= maphigh) 
                & (dfl_next.StreetName.to_numpy()[:, None] == gbf.STNAME.to_numpy())
                & (dfl_next.StreetNameSuffix.to_numpy()[:, None] == gbf.STTYPE.to_numpy())
                & (dfl_next.StreetNamePostDirectional.to_numpy()[:, None] == gbf.STSUFDIR.to_numpy())
                & (dfl_next.PLACE.to_numpy()[:, None] == gbf.PLACO1.to_numpy())
                & (dfl_next.STFIPS.to_numpy()[:, None] == gbf.STCOLEFT.to_numpy()))

df_range_l = df_range_l.append(pd.DataFrame(
    np.column_stack([dfl_next.values[i], gbf.values[j]]),
    columns=dfl_next.columns.append(gbf.columns)).set_index(['New_ID']))


#Appending the two sides of the street into a single dataframe again
df_geocode = df_range_l.append(df_range_r).sort_index()


#Print to see if it looks as expected (omit if dataframe is too large to print efficiently)
print(df_geocode.to_string())



#Check if there are unmatched records:
print(df[~df.index.isin(df_geocode.index)].to_string())


#---- Here is the start of how to append all the records from the original df that are not in the current df:
# Reference: https://stackoverflow.com/questions/44367672/best-way-to-join-merge-by-range-in-pandas


#df_geocode = df_geocode.append(
#    df[~np.in1d(np.arange(len(df)), np.unique(i))],
#    ignore_index=False, sort=False
#)


ModuleNotFoundError: No module named 'pandas_usaddress'